Import libraries

In [63]:
import numpy as np
import pandas as pd
import random as rd
import math
from random import randint
import operator
import matplotlib.pyplot as plt

In [64]:
def create_items(items_num,weight_lower,weight_upper,price_lower,price_upper):
  """Randomly create the item list"""
  weights = []
  prices = []
  for i in range(items_num):
    weights.append(randint(weight_lower,weight_upper))
    prices.append(randint(price_lower,price_upper))
  d = {'weights':weights,'prices':prices}
  item_list = pd.DataFrame(d)
  return item_list


In [65]:
def initial_pop(item_list, individuals_num):
  """Randomly create the initial population
  0 means the item with the corresponding index was not selected
  if individuals_num is an odd number the function will add one to make it even"""
  if individuals_num%2 != 0:
    individuals_num = individuals_num+1
  initial_pop = np.random.randint(0,2,(individuals_num,item_list.shape[0]))
  return initial_pop

In [66]:
def cal_fitness(population,item_list, max_weight):
  fitnesses = []
  for i in range(population.shape[0]):
    selected_items_weights = sum(population[i]*item_list['weights'])
    if selected_items_weights > max_weight:
      fitnesses.append(0)
    else:
      gain = sum(population[i]*item_list['prices'])
      fitnesses.append(gain)
  return fitnesses

In [67]:
def select_parents(population,fitness_results,ratio):
  number_of_parents = math.ceil(population.shape[0]*ratio)
  #add one to number_of_parents if odd
  if number_of_parents%2 != 0:
    number_of_parents = number_of_parents + 1
  #retrieve the indexes of each fitness value
  indexed = list(enumerate(fitness_results))
  #retieve the top x indexes and values with the heighest fitnesses
  top = sorted(indexed, key=operator.itemgetter(1))[-number_of_parents:]
  #extract only the number of indixes
  top_indexes = list(reversed([i for i, v in top]))
  parents = [population[i] for i in top_indexes]
  parents = np.array(parents)
  return parents

In [68]:
def crossover(parents,crossover_point_ratio):
  """Perform one-point crossover and generate a number of offsprings equal to the number of parents
  crossover_point_ratio determine the crossing point, if the solution have 10 genes and the ratio is 0.5 then the crossing point is 5
  """ 
  offsprings = []
  crossing_point = math.ceil(parents.shape[1]*crossover_point_ratio)
  for i in range(0,parents.shape[0],2):
    offspring1 = np.concatenate((parents[i][0:crossing_point] , parents[i+1][crossing_point:parents.shape[1]]),axis = 0)
    offspring2 = np.concatenate((parents[i+1][0:crossing_point] , parents[i][crossing_point:parents.shape[1]]),axis = 0)
    offsprings.append(list(offspring1))
    offsprings.append(list(offspring2))
  return offsprings

In [69]:
def mutation(offsprings, mutation_degree):
  offsprings = np.array(offsprings)
  for i in range(offsprings.shape[0]):
    random = rd.random()
    if random > mutation_degree:
      continue
    random_position = randint(0,offsprings.shape[1]-1)
    if offsprings[i][random_position] == 0:
      offsprings[i][random_position] = 1
    else:
      offsprings[i][random_position] = 0
  return offsprings


In [70]:
def get_the_fittest_individual(population, item_list, max_weight):
  fitness_results = cal_fitness(population,item_list, max_weight)
  #retrieve the indexes of each fitness value
  indexed = list(enumerate(fitness_results))
  #retieve the top 1 index and value with the heighest fitnesses
  top = sorted(indexed, key=operator.itemgetter(1))[-1:]
  #extract the top individual
  top_indexes = list(reversed([i for i, v in top]))
  top_individual = [population[i] for i in top_indexes]
  return top_individual

In [71]:
def genetic_algorithm(population,item_list, max_weight, selection_ratio, generation_num, crossover_point_ratio,mutation_degree):
  for i in range(generation_num):
      if i%10 ==0:
        print('Generation: ', i,'\n')
        print(population)
        print('-'*100)

      #call fitness function on the population
      fitness_results = cal_fitness(population,item_list, max_weight)

      #select parents from population based on the fitness results and the selection ratio specified
      parents = select_parents(population,fitness_results,selection_ratio)

      #crossover parents using one-point crossong to generate new offsprings
      offsprings = crossover(parents,crossover_point_ratio)

      #perform random mutations on offsprings
      mutants = mutation(offsprings, mutation_degree)

      #compine parents and mutants to form the new population in the next generation
      population = np.concatenate((parents,mutants),axis = 0)
  
  #get the fittest individual in the last generation
  fittest_individual = get_the_fittest_individual(population, item_list, max_weight)
  print('Fittest individual: ',fittest_individual,'\n' ,'Fitness: ',cal_fitness(np.array(fittest_individual),item_list, max_weight))

In [72]:
item_list = create_items(100,1,10,40,100)
item_list.weights.sum()
item_list

,weights,prices
0,2,54
1,6,62
2,3,91
3,8,54
4,3,42
...,...,...
95,3,86
96,1,74
97,4,73
98,2,99


In [73]:
population = initial_pop(item_list,100)
population.shape

(100, 100)

In [74]:
max_weight = 300
selection_ratio = 0.5
generation_num = 300
crossover_point_ratio = 0.5
mutation_degree = 0.9

In [75]:
genetic_algorithm(population,item_list, max_weight, selection_ratio, generation_num, crossover_point_ratio,mutation_degree)

Generation:  0 

[[0 1 1 ... 1 1 1]
 [1 0 1 ... 1 1 0]
 [0 1 0 ... 1 0 1]
 ...
 [0 0 1 ... 0 1 0]
 [1 1 1 ... 1 1 1]
 [0 0 0 ... 1 0 0]]
----------------------------------------------------------------------------------------------------
Generation:  10 

[[1 0 1 ... 0 1 1]
 [1 0 1 ... 1 1 1]
 [1 0 1 ... 0 1 1]
 ...
 [1 1 1 ... 0 1 1]
 [1 0 1 ... 1 1 1]
 [1 0 1 ... 0 1 1]]
----------------------------------------------------------------------------------------------------
Generation:  20 

[[1 0 1 ... 0 1 1]
 [1 0 1 ... 0 1 1]
 [1 0 1 ... 1 1 1]
 ...
 [1 0 1 ... 0 1 1]
 [1 0 1 ... 0 0 1]
 [1 0 1 ... 0 1 1]]
----------------------------------------------------------------------------------------------------
Generation:  30 

[[1 0 1 ... 0 1 1]
 [1 0 1 ... 0 1 1]
 [1 0 1 ... 0 1 1]
 ...
 [1 0 1 ... 0 1 1]
 [1 0 1 ... 0 1 1]
 [1 0 1 ... 0 1 1]]
----------------------------------------------------------------------------------------------------
Generation:  40 

[[1 0 1 ... 0 1 1]
 [1 0 1 